In [1]:
import pandas as pd
import json

# Load CSV files
all_services_price_history = pd.read_csv('data/all_services_price_history.csv')
top_services_all_content = pd.read_csv('data/top_services_all_content.csv')
netflix_all_content = pd.read_csv('data/netflix_all_content.csv')
disney_all_content = pd.read_csv('data/disney_all_content.csv')
hulu_all_content = pd.read_csv('data/hulu_all_content.csv')
prime_all_content = pd.read_csv('data/prime_all_content.csv')
top_services_movies = pd.read_csv('data/top_services_movies.csv')

# Load JSON database files
with open('data/streaming_pricing_db.json', 'r') as file:
    streaming_pricing_db = json.load(file)

with open('data/streaming_content_db.json', 'r') as file:
    streaming_content_db = json.load(file)

with open('data/cabel_tv_db.json', 'r') as file:
    cable_tv_db = json.load(file)

In [2]:
import altair as alt

# Convert the Date column to datetime if not already
all_services_price_history['Date'] = pd.to_datetime(all_services_price_history['Date'])

# Create the line chart
price_trends_chart = alt.Chart(all_services_price_history).mark_line().encode(
    x='Date:T',
    y='Subscription Price:Q',
    color='Streaming Service:N',
    tooltip=['Streaming Service', 'Date', 'Subscription Price']
).properties(
    title='Subscription Price Trends Over Time',
    width=800,
    height=400
)

price_trends_chart

/var/folders/6p/h0sr07r51_qcbxx4mdpg2cc80000gn/T/ipykernel_36956/791036149.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_services_price_history['Date'] = pd.to_datetime(all_services_price_history['Date'])
/Users/lanahuyen/anaconda3/envs/dev/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [3]:
# Sum content counts for each streaming service
content_counts = top_services_all_content[['Netflix', 'Hulu', 'Prime Video', 'Disney+']].sum().reset_index()
content_counts.columns = ['Streaming Service', 'Number of Titles']

# Bar chart
content_chart = alt.Chart(content_counts).mark_bar().encode(
    x='Streaming Service:N',
    y='Number of Titles:Q',
    color='Streaming Service:N',
    tooltip=['Streaming Service', 'Number of Titles']
).properties(
    title='Total Content Available on Each Platform',
    width=600,
    height=400
)

content_chart

/Users/lanahuyen/anaconda3/envs/dev/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [7]:
# Check the column names of top_services_all_content
print(top_services_all_content.columns)

Index(['Unnamed: 0', 'Title', 'Year', 'Age', 'Rotten Tomatoes', 'Netflix',
       'Hulu', 'Prime Video', 'Disney+'],
      dtype='object')


In [9]:
# Find the latest subscription price for each streaming service
latest_prices = all_services_price_history.sort_values('Date').groupby('Streaming Service').tail(1)[['Streaming Service', 'Subscription Price']]

# Merge with the average quality DataFrame
pricing_quality_data_latest = latest_prices.merge(avg_quality_df, on='Streaming Service')

# Scatter plot using Altair
quality_price_chart_latest = alt.Chart(pricing_quality_data_latest).mark_circle(size=100).encode(
    x='Subscription Price:Q',
    y='Average Quality:Q',
    color='Streaming Service:N',
    tooltip=['Streaming Service', 'Subscription Price', 'Average Quality']
).properties(
    title='Latest Subscription Price vs. Average Content Quality',
    width=600,
    height=400
)

quality_price_chart_latest

/Users/lanahuyen/anaconda3/envs/dev/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [10]:
# Calculate the average subscription price over time for each streaming service
average_prices = all_services_price_history.groupby('Streaming Service')['Subscription Price'].mean().reset_index()

# Merge with the average quality DataFrame
pricing_quality_data_avg = average_prices.merge(avg_quality_df, on='Streaming Service')

# Scatter plot using Altair
quality_price_chart_avg = alt.Chart(pricing_quality_data_avg).mark_circle(size=100).encode(
    x='Subscription Price:Q',
    y='Average Quality:Q',
    color='Streaming Service:N',
    tooltip=['Streaming Service', 'Subscription Price', 'Average Quality']
).properties(
    title='Average Subscription Price vs. Average Content Quality',
    width=600,
    height=400
)

quality_price_chart_avg

/Users/lanahuyen/anaconda3/envs/dev/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [16]:
import altair as alt

# Connected scatter plot
connected_scatter = alt.Chart(pricing_quality_data_time).mark_line(point=True).encode(
    x='Subscription Price:Q',
    y='Average Quality:Q',
    color='Streaming Service:N',
    tooltip=['Streaming Service', 'Date:T', 'Subscription Price', 'Average Quality']
).properties(
    title='Change in Subscription Price and Content Quality Over Time',
    width=800,
    height=400
).encode(
    alt.X('Subscription Price:Q', scale=alt.Scale(domain=[pricing_quality_data_time['Subscription Price'].min(), pricing_quality_data_time['Subscription Price'].max()])),
    alt.Y('Average Quality:Q', scale=alt.Scale(domain=[0, 100])),
).encode(
    alt.Order('Date:T')
)

connected_scatter

/Users/lanahuyen/anaconda3/envs/dev/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [5]:
# Load cable TV data into a DataFrame
cable_tv_data = pd.DataFrame(cable_tv_db)
cable_tv_data['Date'] = pd.to_datetime(cable_tv_data['Date'])

# Streaming service growth chart
streaming_growth_chart = alt.Chart(all_services_price_history).mark_line().encode(
    x='Date:T',
    y='Subscription Price:Q',
    color='Streaming Service:N',
    tooltip=['Streaming Service', 'Date', 'Subscription Price']
).properties(
    width=800,
    height=200
)

# Cable TV subscribers chart
cable_tv_chart = alt.Chart(cable_tv_data).mark_line(color='blue').encode(
    x='Date:T',
    y='Subscribers:Q',
    tooltip=['Date', 'Subscribers']
).properties(
    width=800,
    height=200
)

# Combine the charts vertically
combined_chart = alt.vconcat(streaming_growth_chart, cable_tv_chart).resolve_scale(
    y='independent'
).properties(
    title='Streaming Services vs. Cable TV Trends'
)

combined_chart

KeyError: 'Date'

In [6]:
# Example interactive chart for Netflix content by year
netflix_chart = alt.Chart(netflix_all_content).mark_bar().encode(
    x='Year:O',
    y='count()',
    tooltip=['Year', 'count()']
).properties(
    title='Netflix Content Added by Year',
    width=600,
    height=400
)

netflix_chart

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

See https://altair-viz.github.io/user_guide/large_datasets.html for information on how to plot large datasets, including how to install third-party data management tools and, in the right circumstance, disable the restriction

alt.Chart(...)